## Employment of all CPU Cores 

In [ ]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

## Importation of Libraries and Definition of Essential Functions

### Libraries

In [ ]:
# For Storing Data AND Creating and Animating Graphs
import collections
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt, mpld3
from matplotlib.animation import FuncAnimation
from matplotlib import figure
from IPython.display import display, clear_output

# For Creating Python Dashboard
import operator as op
import requests
import param
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import hvplot.streamz
import holoviews as hv
from holoviews.element.tiles import EsriImagery
from holoviews.selection import link_selections
from datashader.utils import lnglat_to_meters
from streamz.dataframe import PeriodicDataFrame
from ipywidgets import interact

# Other
import datetime as dt
import serial
import time
import sys
import math
import csv
import psutil

# System Parameters
delay = 0.001

### Functions

In [ ]:
def collect_data(ser, t0):
    time.sleep(delay)                    # delay of 1ms
    val = ser.readline()                # read complete line from serial output
    while not '\\n'in str(val):         # check if full data is received. 
        # This loop is entered only if serial read value doesn't contain \n
        # which indicates end of a sentence. 
        # str(val) - val is byte where string operation to check `\\n` 
        # can't be performed
        time.sleep(delay)                # delay of 1ms 
        temp = ser.readline()           # check for serial output.
        if not not temp.decode():       # if temp is not empty.
            val = (val.decode()+temp.decode()).encode()
            # requrired to decode, sum, then encode because
            # long values might require multiple passes
    val = val.decode()                  # decoding from bytes
    val = val.strip()                   # stripping leading and trailing spaces.
    return [val, round(time.time() - t0, 3)]

In [ ]:
# Text Processing to extract data into arrays

def process_data(data, num_regions):
    
    np_angles = np.zeros([1,num_regions])
    np_bendlabs = np.zeros([1, 2])
    np_time = np.zeros([1, 1])
    

    for i in range(len(data)):

        angles = data[i][0].split("|")[0][1:6].split(",")
        for j in range(len(angles)): angles[j] = int(angles[j])

        bendlabs = data[i][0].split("|")[1].strip("()").split("  ")
        for j in range(len(bendlabs)): bendlabs[j] = float(bendlabs[j])

        np_angles = np.row_stack((np_angles, np.array(angles)))
        np_bendlabs = np.row_stack((np_bendlabs, np.array(bendlabs)))
        np_time = np.row_stack((np_time, np.array(data[i][1])))

    np_angles = np.delete(np_angles, 0, 0)
    np_bendlabs = np.delete(np_bendlabs, 0, 0)
    np_time = np.delete(np_time, 0, 0)

    processed_data = [np_angles, np_bendlabs, np_time]         
    return processed_data
                                       
    # except:
    #     print("Failed on ", i, data[i])

In [ ]:
#function takes in bend position and angle
def opensensor(x, a):
    thickness = 0.002
    length = 7.5
    suppangle = 180 - a
    #print(suppangle)
    smalla = 90 - a/2
    #print(smalla)
    c = math.sqrt(2*length**2 - 2*length*length*math.cos(a*(math.pi/180)))
    #print(c)
    theta = (180-a)/2
    radius = (0.5*c)/math.sin(theta*(math.pi/180))
    strain = (0.5*thickness/radius)
    return [x, radius, strain] 

#function takes in bend position and angle
def closesensor(x, a):
    radius = -length/(math.pi*(1-(a/180)))
    thickness = 0.003 #in meters
    strain = (0.5*thickness/radius) 
    return [x, radius, strain]

#function takes in bendlabs percent strain and outputs strain and stress
def blss(s):
    strain = s/100 
    E = 3600000 #in Pa
    stress = strain*E
    return [strain, stress]

## Serial Data Reading, Cleaning, and Preparation 

In [ ]:
# make sure the 'COM#' is set according the Windows Device Manager
ser = serial.Serial('/dev/tty.usbmodem14401', 9600, timeout=1)
time.sleep(2)
print("connected to: " + ser.portstr)

In [ ]:
sample = []
t0 = time.time()

for i in range(200):
    sample.append(collect_data(ser, t0))
    

In [ ]:
processed = process_data(sample, 3)
print("Contact Sensor Region-Specific Angle Data:", processed[0][0])
print("BendLabs Sensor Data:", processed[1][0])
print("Time Data:", processed[2][0])

In [ ]:
processed = process_data(sample, 3)
processed

## Data Storage Construction and Wrangling

### Notes: 

- Use binary values instead of strings to describe whether the sensor is open or closed for the sake of efficiency and then after the trial is done, map those values into their corresponding category: 0: close, 1:open
- Use Pivot Tables to easily plot values

## Data Collection 

In [ ]:
processed = process_data(sample, 3)
print("Contact Sensor Region-Specific Angle Data:", processed[0][0])
print("BendLabs Sensor Data:", processed[1][0])
print("Time Data:", processed[2][0])

# BendLabs Sensor Data should contain a third column that stores the net angle so that we could determine sensor 
# orientation,or the type of sensor that would be collecting data (open or closed)

In [ ]:
# Determine Time it takes to execute the program herein
start_time = time.time()

# Principal DataFrames' Creation

    # DataFrame for Contact Sensors
ocsensors = pd.DataFrame({"Sensor Type" : [],
              "Sensor Region" : [] ,
              "Contact Angle" : [] ,
              "Time" : [] })
    
    # DataFrame for BendLabs Sensor 
blsensor = pd.DataFrame({ "Strain" : [] ,
              "Stress" :[] ,
              "Time" :[] } )

# Example of the Live-Simulation Algorithm 
for timestep in range(len(processed[2])):
    for index, contactangle in enumerate(processed[0][timestep], start=1):   # default is zero
        
        # DataFrame for Contact Sensors
        ocsensors_temp = pd.DataFrame({"Sensor Type" : ['Normally Open Sensor' if any((True for i in processed[0][timestep] if i >= 0.0)) == True else 'Normally Closed Sensor'],
              "Sensor Region" : [index],
              "Contact Angle" : [contactangle],
              "Time" : processed[2][timestep] })
    
        # DataFrame for BendLabs Sensor 
        blsensor_temp = pd.DataFrame({ "Strain" : [processed[1][timestep][0]],
              "Stress" : [processed[1][timestep][1]],
              "Time" : processed[2][timestep]} )
        
        ocsensors = pd.concat([ocsensors, ocsensors_temp], ignore_index=True, sort=False)
        blsensor = pd.concat([blsensor, blsensor_temp], ignore_index=True, sort=False)

       # ocsensors = ocsensors.append(ocsensors_temp, ignore_index=True,sort=False)
       # blsensor =  blsensor.append(blsensor_temp, ignore_index=True,sort=False)
        
# Print Time it took to run program above
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
ocsensors[Contact]

In [ ]:
['Normally Open Sensor' if any((True for i in processed[0][timestep] if i >= 0.0)) == True else 'Normally Closed Sensor'] 

## Live-Data Visualization

In [ ]:
# Determine Time it takes to execute the program herein
start_time = time.time()

# Principal DataFrames' Creation

    # DataFrame for Contact Sensors
ocsensors = pd.DataFrame({"Sensor Type" : [],
              "Sensor Region" : [] ,
              "Contact Angle" : [] ,
              "Time" : [] })
    
    # DataFrame for BendLabs Sensor 
blsensor = pd.DataFrame({ "Strain" : [] ,
              "Stress" :[] ,
              "Time" :[] } )

# Example of the Live-Simulation Algorithm 
for timestep in range(len(processed[2])):
    for index, contactangle in enumerate(processed[0][timestep], start=1):   # default is zero
        
        # DataFrame for Contact Sensors
        ocsensors_temp = pd.DataFrame({"Sensor Type" : ['Normally Open Sensor' if processed[1][timestep][1] >= 180 else 'Normally Closed Sensor'],
              "Sensor Region" : [index],
              "Contact Angle" : [contactangle],
              "Time" : processed[2][timestep] })
    
        # DataFrame for BendLabs Sensor 
        blsensor_temp = pd.DataFrame({ "Strain" : [processed[1][timestep][0]],
              "Stress" : [processed[1][timestep][1]],
              "Time" : processed[2][timestep]} )
        
        ocsensors = ocsensors.append(ocsensors_temp, ignore_index=True,sort=False)
        blsensor = blsensor.append(blsensor_temp, ignore_index=True,sort=False)
        
# Print Time it took to run program above
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
cpu

In [ ]:
%matplotlib notebook 


# function to update the data
def my_function(i):
    
    # get data
    cpu.popleft()
    cpu.append(psutil.cpu_percent())
    ram.popleft()
    ram.append(psutil.virtual_memory().percent)
    
    ax.clear()
    ax1.clear()
  
    #rolling window size
    repeat_length = 10

    ax.set_xlim([0,repeat_length])
    ax.set_ylim([0,100])
    
    # plot cpu
    ax.plot(cpu)
    ax.scatter(len(cpu)-1, cpu[-1])
    ax.text(len(cpu)-1, cpu[-1]+2, "{}%".format(cpu[-1]))
    ax.set_ylim(0,100)
   

    # plot memory
    ax1.plot(ram)
    ax1.scatter(len(ram)-1, ram[-1])
    ax1.text(len(ram)-1, ram[-1]+2, "{}%".format(ram[-1]))
    ax1.set_ylim(0,100)



# start collections with zeros
cpu = collections.deque(np.zeros(10))
ram = collections.deque(np.zeros(10))

# define and adjust figure
fig = plt.figure(figsize=(12,6), facecolor='#DEDEDE')
ax = plt.subplot(121)
ax1 = plt.subplot(122)
ax.set_facecolor('#DEDEDE')
ax1.set_facecolor('#DEDEDE')

# animate
ani = FuncAnimation(fig, my_function, interval=1000, blit = False)
plt.show()


## Draft Cell Blocks

In [ ]:
# Below are some quick examples.
# Create conditional DataFrame column by np.where() function.
df['Discount'] = np.where(df['Courses']=='Spark', 1000, 2000)

# Another way to create column conditionally.
df['Discount'] = [1000 if x == 'Spark' else 2000 for x in df['Courses']]

# Create conditional DataFrame column by map() and lambda.
df['Discount'] = df.Courses.map( lambda x: 1000 if x == 'Spark' else 2000)

# Create conditional DataFrame column by np.select() function.
conditions = [
    (df['Courses'] == 'Spark') & (df['Duration'] == '30days'),
    (df['Courses'] == 'Spark') & (df['Duration'] == '35days'),
    (df['Duration'] == '50days')]
choices = [1000, 1050,200]
df['Discount'] = np.select(conditions,choices, default=0)

# Using Dictionary to map new values.
Discount_dictionary ={'Spark' : 1500, 'PySpark' : 800, 'Spark' : 1200}
df['Discount'] = df['Courses'].map(Discount_dictionary)

# Pandas create conditional DataFrame column by dictionary
df['Discount'] = [Discount_dictionary.get(v, None) for v in df['Courses']]

# Using DataFrame.assign() method.
def Courses_Discount(row):
    if row["Courses"] == "Spark":
        return 1000
    else:
        return 2000
df = df.assign(Discount=df.apply(Courses_Discount, axis=1))

# Conditions with multiple rand multiple columns.
def Courses_Discount(row):
    if row["Courses"] == "Spark":
        return 1000
    elif row["Fee"] == 25000:
        return 2000
    else:
        return 0
df = df.assign(Discount=df.apply(Courses_Discount, axis=1))

# Using .loc[] property for single condition.
df.loc[(df['Courses']=="Spark"), 'Discount'] = 1000

# Using loc[] method for Multiple conditions.
df.loc[(df['Courses']=="Spark")&(df['Fee']==23000)|(df['Fee']==25000), 'Discount'] = 1000

# Using DataFrame.apply() method with lambda function.
df['Discount'] = df['Courses'].apply(lambda x: '1000' if x=='Spark' else 1000)

# Pandas create conditional column using mask() method.
# Replace values where the condition is True
df['Discount'] = df['Discount'].mask(df['Courses']=='Spark', other=1000)

# Using where()
df['Discount'] = df['Discount'].where(df['Courses']=='Spark', other=1000)

# Using transform() with a lambda function.
df['Discount'] = df['Courses'].transform(lambda x: 1000 if x == 'Spark' else 2000)

In [ ]:
ocsensors.loc[ocsensors['Sensor Region'] == 2.0]

In [ ]:
['Normally Closed Sensor' if x < 0 else 'Normally Closed Sensor' for x in processed[0][0]]

In [ ]:
['Normally Closed Sensor' if max(processed[0][0]) <= 0 else 'Normally Closed Sensor']

In [ ]:
example = pd.DataFrame({"Sensor Type" : ['Normally Open Sensor'],
              "Sensor Region" : [1],
              "Contact Angle" : [max(processed[0][0])],
              "Position" : [2.3] ,
              "Time" : ['5 OCLOCK'] })
example

In [ ]:
ax = sensorregion_time_angle.plot(ylabel = 'Contact Angle',figsize = (10,6),marker='o')


In [ ]:
y1 = ocsensors.loc[ocsensors['Sensor Region'] == 1.0, 'Contact Angle'].to_numpy().flatten()
y2 = ocsensors.loc[ocsensors['Sensor Region'] == 2.0, 'Contact Angle'].to_numpy().flatten()
y3 = ocsensors.loc[ocsensors['Sensor Region'] == 3.0, 'Contact Angle'].to_numpy().flatten()

list(y1)

In [ ]:
#You may ignore this code block until Chapter 8.
sensorregion_time_angle = ocsensors.pivot_table(index = 'Time', columns = 'Sensor Region',values = 'Contact Angle')
sensorregion_time_angle.head()

In [ ]:
# Determine Time it takes to execute the program herein
start_time = time.time()

# Principal DataFrames' Creation

    # DataFrame for Contact Sensors
ocsensors2 = pd.DataFrame({"Sensor Type" : [],
              "Sensor Region" : [] ,
              "Contact Angle" : [] ,
              "Time" : [] })
    
    # DataFrame for BendLabs Sensor 
blsensor2 = pd.DataFrame({ "Strain" : [] ,
              "Stress" :[] ,
              "Time" :[] } )

# Example of the Live-Simulation Algorithm 
for timestep in range(len(processed[2][0])):
    for index, contactangle in enumerate(processed[0][timestep], start=1):   # default is zero
        
        # DataFrame for Contact Sensors
        ocsensors_temp = pd.DataFrame({"Sensor Type" : ['Normally Open Sensor' if processed[1][timestep][1] >= 180 else 'Normally Closed Sensor'],
              "Sensor Region" : [index],
              "Contact Angle" : [contactangle],
              "Time" : processed[2][timestep] })
    
        # DataFrame for BendLabs Sensor 
        blsensor_temp = pd.DataFrame({ "Strain" : [processed[1][timestep][0]],
              "Stress" : [processed[1][timestep][1]],
              "Time" : processed[2][timestep]} )

        ocsensors2 = ocsensors2.append(ocsensors_temp, ignore_index=True,sort=False)
        blsensor2 = blsensor2.append(blsensor_temp, ignore_index=True,sort=False)
        
# Print Time it took to run program above
print("--- %s seconds ---" % (time.time() - start_time))
